# 3. Modélisation avec SVM (Support Vector Machine)

## 3.1. Introduction

Ce notebook explore l'utilisation d'un autre algorithme classique puissant, le **Support Vector Machine (SVM)**, pour la classification des sentiments.

**Note sur le temps de calcul :** Les SVM peuvent être très lents à entraîner sur de grands jeux de données, en particulier avec des noyaux non linéaires. Pour cette raison, nous travaillerons sur un échantillon plus petit des données (100 000 lignes) pour l'entraînement final.

**Étapes Principales :**
- Chargement et échantillonnage des données.
- Vectorisation avec TF-IDF.
- Entraînement et évaluation du modèle.
- Sauvegarde du modèle et du vectoriseur.

## 3.2. Importation des bibliothèques

In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

## 3.3. Chargement et Préparation des Données

In [ ]:
# Charger le jeu de données équilibré
df_full = pd.read_csv('../data/balanced_subset.csv', encoding='utf-8')

# Prendre un échantillon de 100 000 lignes pour l'entraînement du SVM
df = df_full.sample(n=100000, random_state=42)

print("Aperçu de l'échantillon de données :")
print(df.head())

## 3.4. Vectorisation et Division des Données

In [ ]:
# Initialiser TF-IDF avec un vocabulaire limité à 1000 features pour réduire la complexité
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

# Transformer les commentaires en vecteurs
X = vectorizer.fit_transform(df['cleaned_text'])

# Les ratings sont la cible (y)
y = df['rating']

# Diviser les données en ensembles d'entraînement et de test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Forme de l'ensemble d'entraînement : {X_train.shape}")
print(f"Forme de l'ensemble de test : {X_test.shape}")

## 3.5. Entraînement et Évaluation du Modèle SVM

Nous utilisons un noyau RBF (Radial Basis Function), qui est un choix courant et performant pour les SVM.

In [ ]:
# Initialiser le modèle SVM avec des hyperparamètres courants
# Le noyau 'rbf' est un bon point de départ pour de nombreuses tâches.
svm_model = SVC(kernel='rbf', C=10, gamma='scale', random_state=42, verbose=True)

# Entraînement du modèle sur l'ensemble d'entraînement
print("Début de l'entraînement du modèle SVM...")
svm_model.fit(X_train, y_train)
print("Entraînement terminé.")

# Prédictions sur les données de test
y_pred = svm_model.predict(X_test)

# Évaluation des performances
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrécision du modèle SVM : {accuracy:.4f}")

print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

## 3.6. Sauvegarde du Modèle

Nous sauvegardons le modèle SVM entraîné et le vectoriseur correspondant.

In [ ]:
# Sauvegarder le modèle et le vectorizer
joblib.dump(svm_model, '../models/svm/svm_model.pkl')
joblib.dump(vectorizer, '../models/svm/tfidf_vectorizer.pkl')

print("Modèle SVM et vectoriseur sauvegardés dans '../models/svm/'")

## 3.7. Test de Chargement et Prédiction

In [ ]:
# Charger le modèle et le vectorizer
loaded_model = joblib.load('../models/svm/svm_model.pkl')
loaded_vectorizer = joblib.load('../models/svm/tfidf_vectorizer.pkl')

# Nouveau commentaire pour le test
new_comment = ["The experience was just okay, not great but not bad either."]
new_comment_tfidf = loaded_vectorizer.transform(new_comment)

# Prédiction
predicted_rating = loaded_model.predict(new_comment_tfidf)
print(f"Texte : '{new_comment[0]}' -> Prédiction : {predicted_rating[0]}")